In [9]:
import os
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, Model
import pathlib
from tensorflow.keras import Input
import numpy as np
from tensorflow.keras.layers import Lambda, Conv2D, BatchNormalization, MaxPooling2D, Activation, Conv2DTranspose, concatenate, Concatenate
from tensorflow.keras.metrics import MeanIoU

In [15]:
def process_path(image_path, mask_path):
    image = tf.io.read_file(image_path)
    mask = tf.io.read_file(mask_path)
    image = tf.image.decode_png(image, channels = 3)
    mask = tf.image.decode_png(mask, channels = 1)/255
    return image, mask

In [3]:
def batch_data (path_X, path_y, batch_size):
    X_names = os.listdir(path_X)
    X_path = [f'{path_X}/{file}' for file in X_names]
    y_names = os.listdir(path_y)
    y_path = [f'{path_y}/{file}' for file in y_names]
    ds_train = tf.data.Dataset.from_tensor_slices((X_path, y_path))
    return ds_train.map(process_path).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [4]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, (3,3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [5]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters) #can be used as skip connection 
    p = MaxPooling2D((2,2))(x)
    return x, p

In [6]:
def decoder_block(inputs, skip_features, num_filters): #skip features are going to be the x returned from the encoder block
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [7]:
def build_unet(img_height, img_width, channels):
    
    inputs = Input((img_height, img_width, channels))
    inputs = Lambda(lambda x: x / 255)(inputs) #Normalize the pixels by dividing by 255

    #Encoder - downscaling (creating features)
    skip1, pool1 = encoder_block(inputs, 16)
    skip2, pool2 = encoder_block(pool1, 32) 
    skip3, pool3 = encoder_block(pool2, 64)
    skip4, pool4 = encoder_block(pool3, 128) 
    
    #Bottleneck or bridge between encoder and decoder
    b1 = conv_block(pool4, 256)
    
    #Decoder - upscaling (reconstructing the image and giving it precise spatial location)
    decoder1 = decoder_block(b1, skip4, 128)
    decoder2 = decoder_block(decoder1, skip3, 64)
    decoder3 = decoder_block(decoder2, skip2, 32)
    decoder4 = decoder_block(decoder3, skip1, 16)
    
    #Output
    outputs = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(decoder4)
    model = Model(inputs, outputs)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    model.summary()
    
    return model

In [10]:
model = build_unet(256, 256, 3)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 16  448         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 256, 256, 16  64         ['conv2d_1[1][0]']               
 rmalization)                   )                                                             

In [13]:
path_X = '/home/zhlj/code/solar_panel_segmentation/tests/Small_dataset/Images'
path_y = '/home/zhlj/code/solar_panel_segmentation/tests/Small_dataset/Target'
batch_size = 8
dataset = batch_data(path_X=path_X, path_y=path_y, batch_size = batch_size)

In [14]:
model.fit(dataset, epochs = 50, verbose=1)

Epoch 1/50


2022-12-01 13:38:08.802730: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8700
2022-12-01 13:38:10.260794: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


276/276 [==============================] - 59s 186ms/step - loss: -272.3533 - accuracy: 0.5802
Epoch 2/50
276/276 [==============================] - 49s 179ms/step - loss: -736.1082 - accuracy: 0.3296
Epoch 3/50
276/276 [==============================] - 49s 178ms/step - loss: -1463.9888 - accuracy: 0.0444
Epoch 4/50
276/276 [==============================] - 49s 179ms/step - loss: -2568.7461 - accuracy: 1.3178e-04
Epoch 5/50
276/276 [==============================] - 49s 179ms/step - loss: -4093.0784 - accuracy: 3.3749e-05
Epoch 6/50
276/276 [==============================] - 49s 178ms/step - loss: -5917.4458 - accuracy: 7.1900e-06
Epoch 7/50
276/276 [==============================] - 49s 179ms/step - loss: -7998.9604 - accuracy: 1.4532e-07
Epoch 8/50
276/276 [==============================] - 49s 178ms/step - loss: -10445.6064 - accuracy: 0.0000e+00
Epoch 9/50
136/276 [=============>................] - ETA: 24s - loss: -12192.9180 - accuracy: 0.0000e+00

KeyboardInterrupt: 